<a href="https://colab.research.google.com/github/HassanSamkari/Applied-Data-Science-Capstone/blob/master/Fashion_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [0]:
sales = pd.read_csv('/content/drive/My Drive/project for colab/data/sales_stock.csv')
products = pd.read_csv('/content/drive/My Drive/project for colab/data/products.csv')
positions = pd.read_csv('/content/drive/My Drive/project for colab/data/positions.csv')


In [39]:
sales.head()

,date_number,product_id,color_id,size_id,sales,stock
0,0,310130,78467031,7026791,1,93
1,0,310130,78467031,8015276,2,62
2,0,310130,78467031,452052101,1,18
3,0,310130,78467031,541898038,2,42
4,0,310130,78467031,558685710,3,113


In [40]:
products.head()

,product_id,family_id,subfamily_id,price
0,151926,396066037,335531561,25.95
1,213413,552529755,11509337,19.95
2,310130,679611953,533441312,12.95
3,373883,775013441,62518413,17.95
4,455200,998145072,490222156,29.95


In [41]:
positions.head()

,date_number,product_id,category_id,position
0,0,4450020,4461548,17
1,0,42147334,4461548,4
2,0,81131830,4461548,35
3,0,84035833,4461548,38
4,0,125252584,4461548,39


In [42]:
sales_df=pd.merge(products, sales, left_on='product_id', right_on='product_id')
sales_df.head()

,product_id,family_id,subfamily_id,price,date_number,color_id,size_id,sales,stock
0,151926,396066037,335531561,25.95,59,909958217,111455099,0,17
1,151926,396066037,335531561,25.95,59,909958217,242364222,3,63
2,151926,396066037,335531561,25.95,59,909958217,513017386,1,33
3,151926,396066037,335531561,25.95,59,909958217,820445075,1,42
4,151926,396066037,335531561,25.95,60,909958217,54666687,2,72


In [43]:
stay_columns=['product_id','family_id','subfamily_id','price'] 
sales_final_df = pd.DataFrame(sales_df.groupby(stay_columns).sum()['sales']).reset_index()
sales_final_df.loc[:,'stock']=pd.DataFrame(sales_df.groupby(stay_columns).sum()[ 'stock']).reset_index()['stock']
sales_final_df.head()

,product_id,family_id,subfamily_id,price,sales,stock
0,151926,396066037,335531561,25.95,167,7995
1,213413,552529755,11509337,19.95,757,56646
2,310130,679611953,533441312,12.95,578,49639
3,455200,998145072,490222156,29.95,24,2258
4,571044,831347344,750943270,15.95,591,18868


In [0]:
positions.loc[:,'high_rank']=positions.loc[:,'position'] 
high_rank=pd.DataFrame(positions.groupby(['date_number','product_id']).min()['position'] ).reset_index()
positions=positions.drop('high_rank', axis=1)

In [0]:
positions.loc[:,'low_rank']=positions.loc[:,'position'] 
low_rank=pd.DataFrame(positions.groupby(['date_number','product_id']).max()['position'] ).reset_index()
positions=positions.drop('low_rank', axis=1)

In [0]:
positions.loc[:,'rank_mean']=positions.loc[:,'position'] 
rank_mean=pd.DataFrame(positions.groupby(['date_number','product_id']).mean()['position']).reset_index()
rank_mean=round(rank_mean)
positions=positions.drop('rank_mean', axis=1)

In [0]:
positions=rank_mean.copy().rename(columns={"position": "rank_mean"}) 
positions.loc[:,'high_rank']=high_rank['position'] 
positions.loc[:,'low_rank']=low_rank['position'] 

In [49]:
df=pd.merge(sales_final_df, positions,  left_on=['product_id'], right_on=['product_id']) 
df.head()

,product_id,family_id,subfamily_id,price,sales,stock,date_number,rank_mean,high_rank,low_rank
0,151926,396066037,335531561,25.95,167,7995,59,36.0,9,62
1,151926,396066037,335531561,25.95,167,7995,60,37.0,10,64
2,151926,396066037,335531561,25.95,167,7995,61,38.0,10,65
3,151926,396066037,335531561,25.95,167,7995,62,38.0,10,65
4,151926,396066037,335531561,25.95,167,7995,63,38.0,10,63


In [0]:
train = df.loc[df['date_number'] <= 71].reset_index(drop=True)
test = df.loc[df['date_number'] >= 72].reset_index(drop=True)

In [51]:
train.head()

,product_id,family_id,subfamily_id,price,sales,stock,date_number,rank_mean,high_rank,low_rank
0,151926,396066037,335531561,25.95,167,7995,59,36.0,9,62
1,151926,396066037,335531561,25.95,167,7995,60,37.0,10,64
2,151926,396066037,335531561,25.95,167,7995,61,38.0,10,65
3,151926,396066037,335531561,25.95,167,7995,62,38.0,10,65
4,151926,396066037,335531561,25.95,167,7995,63,38.0,10,63


In [0]:
X_train=train.drop(['sales'], axis=1) 
y_train=train.loc[:,'sales']

In [0]:
X_test=test.drop(['sales'], axis=1) 
y_train=test.loc[:,'sales']